<a href="https://colab.research.google.com/github/Uthmanic/HAMOYE-DataScience-Internship/blob/master/Hamoye_Stage_D(Planet_Understanding_the_Amazon_from_SpaceAmazon).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install kaggle
!pip install kaggle

In [2]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [3]:
import json
token = {"username":"tijaniuthman","key":"42a091ba934aa0b6d9ef884eb7f52567"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [4]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [5]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [6]:
!chmod 600 /root/.kaggle/kaggle.json

In [7]:
#search for the dataset 
!kaggle datasets list -s planets-dataset

ref                                               title                                     size  lastUpdated          downloadCount  
------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  
nikitarom/planets-dataset                         planets_dataset                            1GB  2020-02-09 22:01:31           1950  
mrisdal/open-exoplanet-catalogue                  Open Exoplanet Catalogue                 123KB  2017-06-08 19:59:22           2805  
eduardowoj/exoplanets-database                    Exoplanets Database                      281KB  2018-02-28 20:56:05            852  
muhakabartay/markmarkohkeplerconfirmedplanets     Kepler Confirmed Planets                 521KB  2019-12-26 20:08:20             17  
nasa/kepler-exoplanet-search-results              Kepler Exoplanet Search Results            1MB  2017-10-10 18:26:59           4014  
jsphyg/star-wars                                  Star 

In [8]:
#download dataset
!kaggle datasets download -d nikitarom/planets-dataset -p /content

planets-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
#unzip the dataset
%%capture
!unzip \*.zip

In [10]:
#Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
import cv2
from PIL import Image
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import fbeta_score
import time
%matplotlib inline

pal = sns.color_palette()

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [11]:
#check present working directory
! pwd

/content


In [12]:
#read the train classes csv file and save a copy
df_train_classes= pd.read_csv('/content/planet/planet/train_classes.csv')
df_train = df_train_classes.copy()
df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [13]:
#read the sample submission classes csv file and save a copy
df_test_classes = pd.read_csv('/content/planet/planet/sample_submission.csv')
df_test = df_test_classes.copy()
df_test.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [14]:
#total train data points
!ls /content/planet/planet/train-jpg | wc -l

40479


In [15]:
test1 = !ls /content/planet/planet/test-jpg | wc -l
test2 = !ls /content/test-jpg-additional/test-jpg-additional | wc -l
assert df_test.shape[0] == float(test1[0])+float(test2[0])

In [16]:
#list of unique labels/tags 
labels = df_train['tags'].apply(lambda x: x.split(' '))
from collections import Counter, defaultdict
counts = defaultdict(int)
for l in labels:
    for l2 in l:
        counts[l2] += 1
               
tag_list=list(counts.keys())
tag_list 

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [17]:
#total unique labels
len(tag_list)

17

In [18]:
#assign numerical value to each label using dictionary
label_map = {i:j for j, i in enumerate(tag_list)}
label_map

{'agriculture': 2,
 'artisinal_mine': 13,
 'bare_ground': 12,
 'blooming': 14,
 'blow_down': 16,
 'clear': 3,
 'cloudy': 9,
 'conventional_mine': 11,
 'cultivation': 7,
 'habitation': 5,
 'haze': 0,
 'partly_cloudy': 10,
 'primary': 1,
 'road': 6,
 'selective_logging': 15,
 'slash_burn': 8,
 'water': 4}

In [19]:
# One hot encode the training labels. Convert the images into pixels and resize them
X_train, Y_train = [], []
for img, label in tqdm(df_train.values, miniters = 1000):
  target = np.zeros(17)
  for tag in label.split(' '):
    target[label_map[tag]]=1
  X_train.append(cv2.resize(cv2.imread('/content/planet/planet/train-jpg/{}.jpg'.format(img)), (64,64)))
  Y_train.append(target)

100%|██████████| 40479/40479 [00:53<00:00, 752.39it/s]


In [20]:
#convert the test images to pixels and resize them as well
X_test=[]
for img, label in tqdm(df_test[:40669].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/planet/planet/test-jpg/{}.jpg'.format(img)), (64,64)))
for img, label in tqdm(df_test[40669:].values, miniters = 1000):
  X_test.append(cv2.resize(cv2.imread('/content/test-jpg-additional/test-jpg-additional/{}.jpg'.format(img)), (64,64)))
x_test = np.array(X_test, np.float16)/255

100%|██████████| 20522/20522 [00:27<00:00, 741.67it/s]


In [21]:
#Confirm the dimensions
len(X_test), len(X_train), len(Y_train)

(61191, 40479, 40479)

In [22]:
import sys
sys.getsizeof(X_train)

361288

In [24]:
#Change lists to numpy arrays and normalize
x_train = np.array(X_train, np.float16)/255
y_train = np.array(Y_train, np.uint8)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [25]:
#delete X_test, X_train, Y_train to free up space
del(X_test, X_train, Y_train)

In [26]:
#Split the data into 5 folds and train on 4 folds while validating on 1 fold

yfull_test = []
yfull_train = []
X_train_, X_val_, Y_train_, Y_val_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 1)

In [28]:
#Build a CNN model
from keras.applications.vgg19 import VGG19
from keras.optimizers import Adam, SGD

input_size = 64
input_channels = 3
batch_size = 64

# Add model parameters including dropout, layers and activation function
base_model = VGG19(include_top=False,
                   weights='imagenet',
                   input_shape=(input_size, input_size, input_channels))

model = Sequential()
model.add(BatchNormalization(input_shape=(input_size, input_size, input_channels)))

model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(17, activation='sigmoid'))

In [30]:
# define model training optimizer parameters
opt  = SGD(lr=0.1)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0),
                ModelCheckpoint(filepath='weights/best_weights',
                                 save_best_only=True,
                                 save_weights_only=True)]
model.summary()




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 64, 3)         12        
_________________________________________________________________
vgg19 (Functional)           (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 17)                34833     
Total params: 20,059,229
Trainable params: 20,059,223
Non-trainable params: 6
_________________________________________________________________


In [31]:
gc.collect()

0

In [32]:
#Try a combination of epoch lengths and learning rates

model.fit(x = X_train_, y= Y_train_, validation_data=(X_val_, Y_val_),batch_size=64,verbose=2, epochs=15,callbacks=callbacks,shuffle=True)

p_val = model.predict(X_val_, batch_size = 64, verbose=2)
print(fbeta_score(Y_val_, np.array(p_val) > 0.2, beta=2, average='samples')) #Check the model performance on the validation set

p_train = model.predict(x_train, batch_size =64, verbose=2) #save the training predictions
yfull_train.append(p_train)
        
p_test = model.predict(x_test, batch_size = 64, verbose=2) #save the test predictions
yfull_test.append(p_test)

Epoch 1/15
405/405 - 42s - loss: 0.1530 - accuracy: 0.7354 - val_loss: 0.1219 - val_accuracy: 0.8478
Epoch 2/15
405/405 - 42s - loss: 0.1164 - accuracy: 0.7277 - val_loss: 0.1053 - val_accuracy: 0.8072
Epoch 3/15
405/405 - 44s - loss: 0.1068 - accuracy: 0.7247 - val_loss: 0.1102 - val_accuracy: 0.7973
Epoch 4/15
405/405 - 44s - loss: 0.1012 - accuracy: 0.7175 - val_loss: 0.1006 - val_accuracy: 0.8044
Epoch 5/15
405/405 - 44s - loss: 0.0963 - accuracy: 0.6998 - val_loss: 0.1027 - val_accuracy: 0.7826
Epoch 6/15
405/405 - 44s - loss: 0.0921 - accuracy: 0.6937 - val_loss: 0.0979 - val_accuracy: 0.7588
Epoch 7/15
405/405 - 44s - loss: 0.0889 - accuracy: 0.7018 - val_loss: 0.0997 - val_accuracy: 0.7865
Epoch 8/15
405/405 - 44s - loss: 0.0840 - accuracy: 0.6645 - val_loss: 0.0998 - val_accuracy: 0.7584
102/102 - 3s
0.9151250790927791
506/506 - 16s
957/957 - 29s


In [33]:
#import and initialise Garbage Collector
import gc
gc.collect()

3693

In [34]:
#make predictions
predictions = model.predict(x_test, batch_size = 32)
predictions

array([[7.58401118e-04, 9.99818623e-01, 5.84853813e-03, ...,
        4.96029258e-02, 8.47792905e-03, 2.68029724e-03],
       [1.63648103e-04, 9.99877334e-01, 8.65130313e-03, ...,
        2.48123825e-01, 4.35863957e-02, 1.34456195e-02],
       [2.05903756e-03, 9.99307990e-01, 4.53616343e-02, ...,
        1.27396523e-03, 9.91400564e-04, 7.35363283e-04],
       ...,
       [1.22860845e-04, 9.99898672e-01, 8.49977415e-03, ...,
        7.42257014e-03, 9.52372607e-03, 7.86441204e-04],
       [5.79984747e-02, 9.31957513e-02, 3.68988723e-03, ...,
        7.89352460e-04, 5.85522386e-04, 4.55373869e-04],
       [1.02320546e-02, 9.24791932e-01, 1.32721186e-01, ...,
        8.51988443e-04, 6.49517961e-03, 8.36013409e-04]], dtype=float32)

In [35]:
#predictions dataframe
pred = pd.DataFrame(predictions, columns =  tag_list)
pred

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,0.000758,0.999819,0.005849,0.995796,0.002214,0.001323,0.002666,0.001711,0.000339,0.000123,0.003655,0.000165,0.000641,0.000064,0.049603,0.008478,0.002680
1,0.000164,0.999877,0.008651,0.997669,0.000682,0.002061,0.002106,0.004936,0.000765,0.000076,0.003972,0.000261,0.000979,0.000120,0.248124,0.043586,0.013446
2,0.002059,0.999308,0.045362,0.002444,0.086951,0.003314,0.036281,0.010482,0.000754,0.000593,0.997619,0.000314,0.000770,0.000138,0.001274,0.000991,0.000735
3,0.001737,0.999750,0.453548,0.985435,0.003371,0.005761,0.006620,0.155872,0.002370,0.000189,0.014865,0.000215,0.005680,0.000170,0.022023,0.003065,0.008659
4,0.009797,0.779058,0.043251,0.006277,0.069778,0.004437,0.025398,0.004390,0.000751,0.198203,0.790184,0.000391,0.003970,0.000855,0.001547,0.000885,0.001218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.003655,0.904809,0.003176,0.004390,0.008031,0.001636,0.009036,0.000505,0.000221,0.101549,0.859027,0.000188,0.000306,0.000085,0.001244,0.000500,0.000427
61187,0.000759,0.998928,0.008694,0.994569,0.999183,0.010213,0.018106,0.002762,0.000352,0.000263,0.002703,0.001562,0.003404,0.001356,0.007713,0.013581,0.000686
61188,0.000123,0.999899,0.008500,0.999570,0.086047,0.001371,0.067706,0.002047,0.000170,0.000027,0.000194,0.000097,0.000611,0.000064,0.007423,0.009524,0.000786
61189,0.057998,0.093196,0.003690,0.039191,0.035902,0.001144,0.004871,0.000904,0.000312,0.862227,0.002721,0.000217,0.001552,0.000160,0.000789,0.000586,0.000455


In [36]:
#Write a loop to assign each value to its predicted label
preds = []
for i in tqdm(range(pred.shape[0]), miniters=1000):
    a = pred.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [02:55<00:00, 347.86it/s]


In [37]:
#Assign the predictions to the tags column of the df_test data frame
df_test['tags'] = preds 
df_test.to_csv('uthman_11th_Submission.csv', index = False) #Export to csv
df_test.head()

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear blooming
2,test_2,primary partly_cloudy
3,test_3,primary agriculture clear
4,test_4,primary partly_cloudy


In [1]:
#download submission file
from google.colab import files
files.download('uthman_11th_Submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>